In [2]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings
import math
from tqdm import tqdm
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
warnings.filterwarnings('ignore')

import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:] #So this dview (direct viewing) becomes the parallelizing program-direct execution in cloud

[0, 1, 2, 3]


In [3]:
import re

In [4]:
einlist = pd.read_csv("/Users/Wolfalev/Documents/Grad/UT-AUSTIN GRAD/Coursework/CUSS/JiRepo/npo_classifier/dataset/eNGO_TX_ein_name_list.csv")

#for ein in einlist['EIN']:
 #   print(ein)

In [5]:
#xml_runner = XMLRunner()
#parsed_filing = xml_runner.run_sked(2012134249347, 'IRS990ScheduleJ')

### Need to get object ID

### Prepare UK Data.

```Python
# Compress icnpo training data.
df_icnpo_classifier_training_data=pd.read_csv('../dataset/icnpo_classifier_training_data.csv', sep=',')
df_icnpo_classifier_training_data.to_pickle('../dataset/icnpo_classifier_training_data.pkl.gzip', compression='gzip')
```

### Prepare US Training Data.
Useful links:
- IRS 990 forms on AWS: https://registry.opendata.aws/irs990/
- NCCS Data Archive (data dict, NTEE codes, etc.): https://nccs-data.urban.org/index.php

### TODO:
- [+] Add year of formation. - Do not use information from 990 forms: only Form 990 has "Year of Formation". Use BMF 199508 to dummy-code the organizations.
- [ ] Add program descriptions in Part VIII of Form 990.

In [6]:
df_index=pd.DataFrame()
for year in range(2013, 2019):
        try:
            print(str(year), 'started', end='\t')
            df_index_temp=pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_'+str(year)+'.csv', 
                                  error_bad_lines=False, warn_bad_lines=True # Be cautious with bad lines.
                                 )
            df_index_temp['YEAR']=year
            df_index=pd.concat([df_index, df_index_temp], ignore_index=True)
            print('got')
        except:
            print('passed')

2013 started	got
2014 started	

b'Skipping line 39569: expected 9 fields, saw 10\n'


got
2015 started	got
2016 started	got
2017 started	got
2018 started	got


In [7]:
df_index.groupby(['YEAR','RETURN_TYPE']).count()

RETURN_ID  FILING_TYPE     EIN  TAX_PERIOD  SUB_DATE  \
YEAR RETURN_TYPE                                                         
2013 990             115531       115531  115531      115531    115531   
     990EO            21991        21991   21991       21991     21991   
     990EZ            60484        60484   60484       60484     60484   
     990O             38029        38029   38029       38029     38029   
     990PF            25414        25414   25414       25414     25414   
2014 990             163381       163381  163381      163381    163381   
     990EO            29466        29466   29466       29466     29466   
     990EZ            82937        82937   82937       82937     82937   
     990O             52490        52490   52490       52490     52490   
     990PF            59254        59254   59254       59254     59254   
2015 990             106571       106571  106571      106571    106571   
     990EO            21399        21399   21399       21399     21399   
     990EZ            59845        59845   59845       59845     59845   
     990O             34567        34567   34567       34567     34567   
     990PF            38650        38650   38650       38650     38650   
2016 990             160550       160550  160550      160550    160550   
     990EO            28537        28537   28537       28537     28537   
     990EZ            84652        84652   84652       84652     84652   
     990O             50987        50987   50987       50987     50987   
     990PF            53694        53694   53694       53694     53694   
2017 990             207734       207734  207734      207734    207734   
     990EO            36318        36318   36318       36318     36318   
     990EZ           109648       109648  109648      109648    109648   
     990O             66766        66766   66766       66766     66766   
     990PF            68547        68547   68547       68547     68547   
2018 990             192489       192489  192489      192489    192489   
     990EO            32690        32690   32690       32690     32690   
     990EZ           103694       103694  103694      103694    103694   
     990O             61005        61005   61005       61005     61005   
     990PF            67632        67632   67632       67632     67632   

                  TAXPAYER_NAME     DLN  OBJECT_ID  
YEAR RETURN_TYPE                                    
2013 990                 115531  115531     115531  
     990EO                21991   21991      21991  
     990EZ                60484   60484      60484  
     990O                 38029   38029      38029  
     990PF                25414   25414      25414  
2014 990                 163381  163381     163381  
     990EO                29466   29466      29466  
     990EZ                82937   82937      82937  
     990O                 52490   52490      52490  
     990PF                59254   59254      59254  
2015 990                 106571  106571     106571  
     990EO                21399   21399      21399  
     990EZ                59845   59845      59845  
     990O                 34567   34567      34567  
     990PF                38650   38650      38650  
2016 990                 160550  160550     160550  
     990EO                28537   28537      28537  
     990EZ                84652   84652      84652  
     990O                 50987   50987      50987  
     990PF                53694   53694      53694  
2017 990                 207734  207734     207734  
     990EO                36318   36318      36318  
     990EZ               109648  109648     109648  
     990O                 66766   66766      66766  
     990PF                68547   68547      68547  
2018 990                 192489  192489     192489  
     990EO                32690   32690      32690  
     990EZ               103694  103694     103694  
     990O                 61005   61005      61005  
     9

### Acquiring text data.

In [8]:
df_index_matched = df_index[df_index['EIN'].isin(einlist['EIN'])]
                       
len(df_index_matched)                                               

2458

In [9]:
df_index_matched.sample(10)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID,YEAR
1667865,15001519,EFILE,752234375,201612,12/4/2017 4:47:08 PM,KEEP ATHENS BEAUTIFUL,990EZ,93492263003117,201712639349200311,2017
1836071,15156484,EFILE,721621543,201706,1/31/2018 6:54:22 AM,CFA SOCIETY OF AUSTIN,990EO,93492313015197,201743139349201519,2018
1152318,13660001,EFILE,262887967,201506,07/28/2016,GLOBAL WILDLIFE CONSERVATION,990,93493120006436,201631209349300643,2016
1254169,13846797,EFILE,201508875,201512,10/05/2016,LANDA GARDENS CONSERVANCY,990,93493137036696,201641379349303669,2016
1729243,14993959,EFILE,746075031,201612,11/30/2017 10:47:06 PM,BAYOU PRESERVATION ASSOCIATION INC,990,93493291007297,201742919349300729,2017
884073,12808738,EFILE,741211684,201412,8/21/2015,SATTERWHITE DUFFY LAKE INC,990EO,93492178007015,201511789349200701,2015
977714,13220520,EFILE,453929881,201412,02/19/2016,MERCURY ONE INC,990,93493320167545,201543209349316754,2016
2184074,16001712,EFILE,751825883,201712,12/13/2018 9:28:50 PM,TEXAS LAND CONSERVANCY,990,93493309019668,201813099349301966,2018
247880,11092223,EFILE,201111056,201212,12/31/2013,TEXAS BOTANICAL GARDENS AND N AMERICAN INTERPR...,990,93493319025663,201313199349302566,2013
334338,11296249,EFILE,942371729,201310,4/7/2014,REMNANT OF ISRAEL,990EZ,93492076002104,201400769349200210,2014


In [10]:
done_index_list=[]
dview.execute('import pandas as pd')
dview.execute('from time import sleep')
dview.execute('from irsx.xmlrunner import XMLRunner')
dview.execute('import os')
dview['xml_runner'] = XMLRunner()
dview['df_index'] = df_index_matched
dview['df_index_text_acq'] = pd.DataFrame()

In [11]:
dview['done_index_list'] = list(set(done_index_list))
dview['counter']=0

In [12]:
###### Define functions ######################################
def func_irs990_text(obj_id):
    IRS990_result=xml_runner.run_sked(obj_id, 'IRS990').result 

    ############## Check filing version.##############
    if IRS990_result!=None:
        IRS990_result=IRS990_result[0]
    else:
        return {'IRS990_p1_ActvtyOrMssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_MssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_DscS': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ##################################################

    ###### IRS990_p1_ActvtyOrMssnDsc #################
    IRS990_p1_ActvtyOrMssnDsc=IRS990_result['schedule_parts']['part_i']['ActvtyOrMssnDsc']
    ##################################################

    ##### IRS990_p3_MssnDsc, IRS990_p3_DscS ##########
    IRS990_p3=IRS990_result['schedule_parts']['part_iii']
    IRS990_p3_MssnDsc='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if key=='MssnDsc'])
    IRS990_p3_DscS='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if (key=='Dsc' or '_Dsc' in key)])
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990_p3.keys() and IRS990_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990_p1_ActvtyOrMssnDsc': IRS990_p1_ActvtyOrMssnDsc,
            'IRS990_p3_MssnDsc': IRS990_p3_MssnDsc,
            'IRS990_p3_DscS': IRS990_p3_DscS,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990ez_text(obj_id):
    IRS990EZ_result=xml_runner.run_sked(obj_id, 'IRS990EZ').result

    ############## Check filing version.##############
    if IRS990EZ_result!=None:
        IRS990EZ_result=IRS990EZ_result[0]
    else:
        return {'IRS990EZ_p3_PrmryExmptPrpsTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ###### IRS990EZ_p3_PrmryExmptPrpsTxt #############
    if 'ez_part_iii' in IRS990EZ_result['schedule_parts'].keys():
        IRS990EZ_p3=IRS990EZ_result['schedule_parts']['ez_part_iii']
        IRS990EZ_p3_PrmryExmptPrpsTxt='##'.join([IRS990EZ_p3[key] for key in IRS990EZ_p3.keys() if key=='PrmryExmptPrpsTxt'])
    else:
        IRS990EZ_p3_PrmryExmptPrpsTxt=''
    ##################################################

    ####### IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt ######
    if 'EZPrgrmSrvcAccmplshmnt' in IRS990EZ_result['groups'].keys():
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt='##'.join([s['DscrptnPrgrmSrvcAccmTxt'] for s in IRS990EZ_result['groups']['EZPrgrmSrvcAccmplshmnt']])
    else:
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt=''
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990EZ_p3.keys() and IRS990EZ_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990EZ_p3_PrmryExmptPrpsTxt': IRS990EZ_p3_PrmryExmptPrpsTxt,
            'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990pf_text(obj_id):
    IRS990PF_result=xml_runner.run_sked(obj_id, 'IRS990PF').result

    ############## Check filing version.##############
    if IRS990PF_result!=None:
        IRS990PF_result=IRS990PF_result[0]
    else:
        return {'IRS990PF_p9a_DscrptnTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990PF_p16b_RltnshpSttmntTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ##### IRS990PF_p9a_DscrptnTxt ##########################
    if 'pf_part_ixa' in IRS990PF_result['schedule_parts'].keys():
        IRS990PF_p9a=IRS990PF_result['schedule_parts']['pf_part_ixa']
        IRS990PF_p9a_DscrptnTxt='##'.join([IRS990PF_p9a[key] for key in IRS990PF_p9a.keys() if 'Txt' in key])
    else:
        IRS990PF_p9a_DscrptnTxt=''
    ########################################################

    ##### IRS990PF_p16b_RltnshpSttmntTxt ###################
    if 'PFRlnOfActyTAccmOfExmptPrps' in IRS990PF_result['groups']:
        IRS990PF_p16b_RltnshpSttmntTxt='##'.join([s['RltnshpSttmntTxt'] for s in IRS990PF_result['groups']['PFRlnOfActyTAccmOfExmptPrps']])
    else:
        IRS990PF_p16b_RltnshpSttmntTxt=''
    ########################################################

    return {'IRS990PF_p9a_DscrptnTxt':IRS990PF_p9a_DscrptnTxt,
            'IRS990PF_p16b_RltnshpSttmntTxt':IRS990PF_p16b_RltnshpSttmntTxt,
           }
###### Define functions ######################################

In [13]:
###### Run main function ################################
df_index_text_acq = pd.DataFrame()
df_index = pd.DataFrame()
df_index = df_index_matched
done_index_list = []
for index in df_index_matched.index:
    try:
        if index not in done_index_list: #Index is position of filing in list of all filings in index file
            obj_id = df_index.loc[index, 'OBJECT_ID'] #if that position is not filled, pull the object id
            return_type=df_index.loc[index, 'RETURN_TYPE'] #if that position is not filled, pull the return type
            if return_type in ['990', '990O']: # (1) if return type  is 990, do the following
                text_dict=func_irs990_text(obj_id) #apply this function for 990s, using the object id from index to get the 990 in a text_dict
                df_index.loc[index, 'IRS990_p1_ActvtyOrMssnDsc']=text_dict['IRS990_p1_ActvtyOrMssnDsc'] #get mission and put it in index file
                df_index.loc[index, 'IRS990_p3_MssnDsc']=text_dict['IRS990_p3_MssnDsc'] # get mssndsc and put it here
                df_index.loc[index, 'IRS990_p3_DscS']=text_dict['IRS990_p3_DscS'] # get DsC and put it here
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']# get schedule 0 and put it here
            elif return_type in ['990EZ', '990EO']: #(2) same thing
                text_dict=func_irs990ez_text(obj_id)
                df_index.loc[index, 'IRS990EZ_p3_PrmryExmptPrpsTxt']=text_dict['IRS990EZ_p3_PrmryExmptPrpsTxt']
                df_index.loc[index, 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']=text_dict['IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']
            elif return_type in ['990PF']: #(3) same thing
                text_dict=func_irs990pf_text(obj_id)
                df_index.loc[index, 'IRS990PF_p9a_DscrptnTxt']=text_dict['IRS990PF_p9a_DscrptnTxt']
                df_index.loc[index, 'IRS990PF_p16b_RltnshpSttmntTxt']=text_dict['IRS990PF_p16b_RltnshpSttmntTxt']
            df_index_text_acq=pd.concat([df_index_text_acq, df_index.loc[[index]]], ignore_index=True) #take the entries we just pulled, turn into df for that row, and concatenate them together into df_index_text_acq
            done_index_list+=[index] ## update that we've done this many indicies. 
    except:
        pass

Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2011v1.3 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.2 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2012v2.3 isn't supported for this operati

In [14]:
len(set(done_index_list)), len(df_index_text_acq)

(2455, 2455)

In [15]:
df_index_text_acq.sample(10)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR
640,93493246008024,742879756,EFILE,NaN,NaN,NaN,NaN,RESEARCH BEING PERFORMED BY VARIOUS ENTITIES (...,WIMBERLEY VALLEY WATERSHED ASSOCIATION (WVWA) ...,POLICY AND ADVOCACY - THE WVWA WORKS WITH REGI...,WIMBERLEY VALLEY WATERSHED ASSOCIATION (WVWA) ...,201422469349300802,12154067,990,12/29/2014,WIMBERLEY VALLEY WATERSHED ASSOCIATION,201312,2014
482,93492219008354,742143423,EFILE,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,NaN,NaN,VERSION_NOT_SUPPORTED,NaN,NaN,NaN,201402199349200835,11894567,990EZ,10/21/2014 4:57:20 PM,KEEP BRAZOS BEAUTIFUL INC,201309,2014
2452,93492319075038,760663372,EFILE,Multi-day seminar presenting papers on environ...,The Produced Water Society is a collection of ...,NaN,NaN,,NaN,NaN,NaN,201833199349207503,16023828,990EZ,12/19/2018 10:42:01 AM,PRODUCED WATER SOCIETY INC,201712,2018
1573,93492350003176,752580047,EFILE,"Education community about recycling, litter ab...","Education community about recycling, litter ab...",NaN,NaN,,NaN,NaN,NaN,201623509349200317,14156776,990EZ,2/17/2017 5:06:43 PM,KEEP IRVING BEAUTIFUL,201609,2017
2111,93493045007328,760567365,EFILE,NaN,NaN,NaN,NaN,,The Peckerwood Gardens Conservation Foundation...,Peckerwood Garden is a repository of rare plan...,The Peckerwood Gardens Conservation Foundation...,201820459349300732,15213911,990,3/12/2018 10:13:59 PM,PECKERWOOD GARDENS CONSERVATION FOUNDATION INC,201703,2018
425,93492128018534,760586373,EFILE,Provided items to beautify the entrance and to...,To support the activities of the Baytown Natur...,NaN,NaN,,NaN,NaN,NaN,201431289349201853,11780394,990EZ,9/19/2014 1:54:08 AM,FRIENDS OF THE BAYTOWN NATURE CENTER INC,201312,2014
2289,93493207000298,462191456,EFILE,NaN,NaN,NaN,NaN,,THE BUILDING CONSERVATION TRUST (BCT) WAS CREA...,THE BUILDING CONSERVATION TRUST'S COASTAL INIT...,THE BUILDING CONSERVATION TRUST (BCT) WAS CREA...,201842079349300029,15724390,990,9/25/2018 5:30:58 AM,BUILDING CONSERVATION TRUST,201712,2018
2292,93493219002398,741780447,EFILE,NaN,NaN,NaN,NaN,,CIVIC ASSOCIATION PROVIDING WATER TO SUBDIVISION,CY FOREST SERVICE ASSOCIATION SUPPLIED WATER T...,CIVIC ASSOCIATION PROVIDING WATER TO SUBDIVISION,201842199349300239,15740059,990O,9/28/2018 8:44:19 AM,CY-FOREST WATER SUPPLY CORPORATION,201712,2018
2016,93493320007157,352234918,EFILE,NaN,NaN,NaN,NaN,,PROMOTING AND DEVELOPING WOMEN'S MINISTRIES. P...,DEVELOPMENT OF WOMEN'S MINISTRIES THROUGH PROG...,PROMOTING AND DEVELOPING WOMEN'S MINISTRIES. P...,201703209349300715,15067455,990,1/2/2018 6:25:46 AM,WOMENS NON-PROFIT ALLIANCE FORMERLY PROVENWAY ...,201612,2018
1529,93492319038996,760668230,EFILE,MAINTAIN PUBLIC DEMONSTRATION GARDENS ON THE G...,HORTICULTURAL TRAINING AND INSTRUCTION,NaN,NaN,,NaN,NaN,NaN,201643199349203899,14148907,990EZ,2/14/2017 7:16:00 PM,FORT BEND COUNTY MASTER GARDENERS INC,201512,2017


In [16]:
#df_index_text_acq_2018= df_index_text_acq[df_index_text_acq.YEAR == 2018]
#All Years

In [17]:
df_index_text_acq.sample(10)
print(len(df_index_text_acq))

2455


len(df_EIN_TXT), len(df_index_matched)

In [18]:
df_bmf_1812=pd.read_csv('https://nccs-data.urban.org/data/bmf/2018/bmf.bm1812.csv')
df_bmf_1712=pd.read_csv('https://nccs-data.urban.org/data/bmf/2017/bmf.bm1712.csv')
df_bmf_1608=pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv')
df_bmf_1512=pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv')
df_bmf_1412=pd.read_csv('https://nccs-data.urban.org/data/bmf/2014/bmf.bm1412.csv')
df_bmf_1312=pd.read_csv('https://nccs-data.urban.org/data/bmf/2013/bmf.bm1312.csv')

In [19]:
df_bmf_1812['YEAR_NCCS'] = 2018
df_bmf_1712['YEAR_NCCS'] = 2017
df_bmf_1608['YEAR_NCCS'] = 2016
df_bmf_1512['YEAR_NCCS'] = 2015
df_bmf_1412['YEAR_NCCS'] = 2014
df_bmf_1312['YEAR_NCCS'] = 2013

#df_bmf_1812['Real_Year'] = df_bmf_1812['TAXPER'].astype(str).str[:4]
#df_bmf_1712['Real_Year'] = df_bmf_1712['TAXPER'].astype(str).str[:4]
#df_bmf_1608['Real_Year'] = df_bmf_1608['TAXPER'].astype(str).str[:4]
#df_bmf_1512['Real_Year'] = df_bmf_1512['TAXPER'].astype(str).str[:4]
#df_bmf_1412['Real_Year'] = df_bmf_1412['TAXPER'].astype(str).str[:4]
#df_bmf_1312['Real_Year'] = df_bmf_1312['TAXPER'].astype(str).str[:4]

#df_bmf_1812 = df_bmf_1812[df_bmf_1812['Real_Year'] == '2018']
#df_bmf_1712 = df_bmf_1712[df_bmf_1712['Real_Year'] == '2017']
#df_bmf_1608 = df_bmf_1608[df_bmf_1608['Real_Year'] == '2016']
#df_bmf_1512 = df_bmf_1512[df_bmf_1512['Real_Year'] == '2015']
#df_bmf_1412 = df_bmf_1412[df_bmf_1412['Real_Year'] == '2014']
#df_bmf_1312 = df_bmf_1312[df_bmf_1312['Real_Year'] == '2013']



In [20]:
print(len(df_bmf_1812))
print(len(df_bmf_1712))
print(len(df_bmf_1608))
print(len(df_bmf_1512))
print(len(df_bmf_1412))
print(len(df_bmf_1312))

1499450
1544575
1584587
1551677
1497172
1412018


In [21]:
#df_bmf_1812.drop_duplicates(subset=['TAXPER'],keep='last')
#df_bmf_1712.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1608.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1512.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1412.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1312.drop_duplicates(subset='EIN',keep='last')

In [22]:
df_bmf_1812.columns

Index(['EIN', 'SEC_NAME', 'FRCD', 'SUBSECCD', 'TAXPER', 'ASSETS', 'INCOME',
       'NAME', 'ADDRESS', 'CITY', 'STATE', 'NTEECONF', 'NTEEFINAL', 'NAICS',
       'ZIP5', 'OUTNCCS', 'OUTREAS', 'RULEDATE', 'FIPS', 'FNDNCD', 'PMSA',
       'MSA_NECH', 'CASSETS', 'CFINSRC', 'CTAXPER', 'CTOTREV', 'ACCPER',
       'RANDNUM', 'NTEECC', 'NTEE1', 'LEVEL4', 'LEVEL1', 'NTMAJ10', 'MAJGRPB',
       'LEVEL3', 'LEVEL2', 'NTMAJ12', 'NTMAJ5', 'FILER', 'ZFILER',
       'YEAR_NCCS'],
      dtype='object')

In [23]:
print(len(df_bmf_1812))
print(len(df_bmf_1712))
print(len(df_bmf_1608))
print(len(df_bmf_1512))
print(len(df_bmf_1412))
print(len(df_bmf_1312))

1499450
1544575
1584587
1551677
1497172
1412018


In [30]:
df_bmf_13_18=pd.concat([df_bmf_1312, df_bmf_1412, df_bmf_1512, df_bmf_1608, df_bmf_1712, df_bmf_1812], ignore_index=True)
df_bmf_13_18['Real_Year'] = df_bmf_13_18['TAXPER'].astype(str).str[:4]
df_bmf_13_18= df_bmf_13_18.drop_duplicates(subset=['EIN', 'Real_Year'],keep='last')
#remove any duplicated taxperiods within an ein, only taking  the last


In [31]:
df_bmf_13_18 = df_bmf_13_18[df_bmf_13_18.STATE == 'TX']
df_bmf_13_18 = df_bmf_13_18[df_bmf_13_18.NTEE1 == 'C']

In [32]:
df_bmf_13_18['EIN'] = pd.to_numeric(df_bmf_13_18['EIN'])
df_index_text_acq['EIN'] = pd.to_numeric(df_index_text_acq['EIN'])

In [33]:
#df_bmf_13_18.drop_duplicates(subset=['EIN', 'CTAXPER'],keep='first')
#print(len(df_bmf_13_18))
df_index_text_acq['Real_Year'] = df_index_text_acq['TAX_PERIOD'].astype(str).str[:4]
df_index_text_acq = df_index_text_acq.drop_duplicates(subset=['EIN', 'Real_Year'],keep='last')

print(len(df_index_text_acq))

2438


In [34]:
#df_bmf_13_18['YEAR_real'] = df_bmf_13_18['CTAXPER']
#df_index_text_acq['YEAR_real'] = df_index_text_acq['TAX_PERIOD']

In [36]:
df_EIN_TXT=df_index_text_acq.merge(df_bmf_13_18, on=['EIN', 'Real_Year'] , how='outer')
df_EIN_TXT = df_EIN_TXT[df_EIN_TXT.Real_Year != 'nan']
#df_EIN_TXT['unique_id'] = df_EIN_TXT['EIN'] + "_" + df_EIN_TXT['Real_Year']
#df_EIN_TXT.drop_duplicates(subset='unique_id', keep='last')
df_EIN_TXT['Real_Year'] = pd.to_numeric(df_EIN_TXT['Real_Year'])
df_EIN_TXT = df_EIN_TXT[df_EIN_TXT.Real_Year > 2012]

In [37]:
df_EIN_TXT.sample(3)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,...,cFinSrc,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5
4397,NaN,203058167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,submaster990n,201412.0,NaN,Y,A,C42,C,EN,EN,OT
3375,NaN,912053912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,bmf1312c3,201305.0,0.0,Y,A,C42,C,EN,EN,OT
4546,NaN,352300551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,submaster990n,201312.0,NaN,Y,A,C42,C,EN,EN,OT


In [38]:
df_EIN_TXT.columns

Index(['DLN', 'EIN', 'FILING_TYPE', 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt',
       'IRS990EZ_p3_PrmryExmptPrpsTxt', 'IRS990PF_p16b_RltnshpSttmntTxt',
       'IRS990PF_p9a_DscrptnTxt', 'IRS990ScheduleO_ExplntnTxt',
       'IRS990_p1_ActvtyOrMssnDsc', 'IRS990_p3_DscS', 'IRS990_p3_MssnDsc',
       'OBJECT_ID', 'RETURN_ID', 'RETURN_TYPE', 'SUB_DATE', 'TAXPAYER_NAME',
       'TAX_PERIOD', 'YEAR', 'Real_Year', 'ACCPER', 'ADDRESS', 'ASSETS',
       'CASSETS', 'CFINSRC', 'CITY', 'CTAXPER', 'CTOTREV', 'EPOST', 'FILER',
       'FIPS', 'FISYR_IMAGE', 'FNDNCD', 'FRCD', 'GEN', 'INCOME', 'IRS990n',
       'LEVEL1', 'LEVEL2', 'LEVEL3', 'LEVEL4', 'MAJGRPB', 'MSA_NECH', 'NAICS',
       'NAME', 'NTEE1', 'NTEECC', 'NTEECONF', 'NTEEFINAL', 'NTEESRC',
       'NTMAJ10', 'NTMAJ12', 'NTMAJ5', 'OUTNCCS', 'OUTREAS', 'PFFRCD', 'PMSA',
       'RANDNUM', 'RULEDATE', 'RandNum', 'SEC_NAME', 'STATE', 'SUBSECCD',
       'TAXPER', 'YEAR_NCCS', 'ZFILER', 'ZIP5', 'cAssets', 'cFiler', 'cFinSrc',
       'cTaxPer', 'cTotRev'

In [39]:
print(len(df_EIN_TXT))

6046


In [40]:
df_EIN_TXT['EIN'].unique

<bound method Series.unique of 0       421602181
3       746046937
13      746049372
16      756423660
20      760691272
21      237357269
23      742121275
27      742645332
28      752212157
29      742094805
35      510225607
36      752154523
106     750787690
116     751954862
144     742601251
148     746069002
178     276402231
180     760436267
194     760567365
195     274374251
198     752351673
204     752856276
208     272887426
258     752655909
259     742772459
274     746059311
294     742687511
295     752856276
296     741854299
297     237109965
          ...    
8822    811919317
8823    812331662
8824    812516850
8825    812901088
8826    820560758
8827    820571299
8828    830434834
8829    830462989
8830    841512096
8831    850284313
8832    861067178
8833    861157331
8835    870754471
8836    900357998
8837    900391541
8838    900453229
8839    900505937
8842    900601557
8843    900770416
8844    900793319
8845    900889748
8846    900986516
8847    9010214

In [41]:
df_EIN_TXT['EIN'].isin(einlist['EIN']).value_counts()

False    3229
True     2817
Name: EIN, dtype: int64

In [ ]:
#df_EIN_TXT[['EIN', 'TAX_PERIOD']].drop_duplicates()
#len(df_EIN_TXT)

In [42]:
df_EIN_TXT.to_csv('./texas_20132018_envorgs.csv', index=False)

In [43]:
df_EIN_TXT_list = df_EIN_TXT[df_EIN_TXT['EIN'].isin(einlist['EIN'])]

In [44]:
len(df_EIN_TXT_list)

2817

In [45]:
df_EIN_TXT_list.to_csv('./texas_20132018_envorgs_matched.csv', index=False)